In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data_train = pd.read_csv('/kaggle/input/playground-series-s4e11/train.csv')
data_test = pd.read_csv('/kaggle/input/playground-series-s4e11/test.csv')
sample_submission = pd.read_csv('/kaggle/input/playground-series-s4e11/sample_submission.csv')

In [ ]:
data_train.head()

In [ ]:
data_train.dtypes

In [ ]:
data_train.isna().sum()

In [ ]:
data_train.Depression.hist()

In [ ]:
data_train[data_train.select_dtypes(exclude=['object']).columns] = data_train.select_dtypes(exclude=['object']).fillna(0)
data_train[data_train.select_dtypes(include=['object']).columns] = data_train.select_dtypes(include=['object']).fillna('None')

X = data_train.drop(['id', 'Name', 'Depression'], axis=1)
y = data_train.Depression

In [ ]:
from sklearn.model_selection import train_test_split
from catboost import Pool

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=123)

train_pool = Pool(X_train, y_train, cat_features=X_train.select_dtypes(include=['object']).columns.values)

val_pool = Pool(X_val, y_val, cat_features=X_train.select_dtypes(include=['object']).columns.values)

In [ ]:
from optuna import create_study
from optuna.samplers import TPESampler
from catboost.utils import eval_metric
from catboost import CatBoostClassifier


def objective(trial):
  params = {
      'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
      'depth': trial.suggest_int('depth', 3, 10),
      'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
      'boosting_type': trial.suggest_categorical('boosting_type', ['Ordered', 'Plain'])
      }

  model = CatBoostClassifier(**params, random_seed=42, task_type='GPU', verbose=0, iterations = 400)
  model.fit(train_pool, eval_set=val_pool)
  y_pred = model.predict_proba(val_pool)
  return eval_metric(val_pool.get_label(), y_pred[:, 1], 'AUC')

sampler = TPESampler(seed=123)
study = create_study(direction='maximize', sampler=sampler)
study.optimize(objective, n_trials=10)

In [ ]:
model = CatBoostClassifier(**study.best_params, random_seed=42, task_type='GPU',)

model.fit(train_pool, verbose=0, eval_set=val_pool)
y_pred = model.predict_proba(val_pool)
eval_metric(val_pool.get_label(), y_pred[:, 1], 'AUC')

In [ ]:
data_test[data_test.select_dtypes(exclude=['object']).columns] = data_test.select_dtypes(exclude=['object']).fillna(0)
data_test[data_test.select_dtypes(include=['object']).columns] = data_test.select_dtypes(include=['object']).fillna('None')

data_test = data_test.drop(['id', 'Name'], axis=1)

y_pred = model.predict(data_test)

sample_submission['Depression'] = y_pred
sample_submission.to_csv('submission.csv', index=False)